In [49]:
import json
import xmltodict
import pandas as pd
import unidecode

In [50]:
# Função para retirar acentuação 
def f(str):
    return (unidecode.unidecode(str))

### A intenção é estudar 136 deputados estaduais. O corte são deputados com despesas de gabinete, ou seja, os 94 eleitos em 2014 e os 42 restantes, que podem ser suplentes que assumiram o cargo em algum período e usaram a verba de gabinete ou nomes da legislatura passada que registram gastos em 2015 e precisarão ser excluídos

### O problema é que o código único que identifica os deputados tem nomes diferentes nas bases de dados abertos (IdMembro, IdAutor etc.)

### Ou nem existe na base de despesas

### A solução encontrada foi criar uma base única com os nomes de deputados que usaram a cota parlamentar na atual legislatura. No final sobrarão 120 nomes

### Carrega arquivo gerado em alesp_despesas

In [61]:
nomes_deputados_atual_legislatura = pd.read_csv('resultados/nomes_deputados_atual_legislatura.csv')

In [62]:
nomes_deputados_atual_legislatura["Deputado"] = nomes_deputados_atual_legislatura["Deputado"].apply(f)

In [63]:
nomes_deputados_atual_legislatura.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 1 columns):
Deputado    136 non-null object
dtypes: object(1)
memory usage: 1.1+ KB


### Agora cruzo com a base do TSE para ter só eleitos e suplentes, gerado em alesp_doacoes

In [64]:
deputados_queconcorreram_2014_sp = pd.read_csv('resultados/deputados_queconcorreram_2014_sp.csv', converters={'sequencial': lambda x: str(x), 'cpf': lambda x: str(x)})

In [65]:
deputados_queconcorreram_2014_sp['nome_urna'] = deputados_queconcorreram_2014_sp['nome_urna'].apply(f)

Seleciona só eleitos e suplentes

VANESSA DAMO foi incluída abaixo porque perdeu o mandato depois de eleita, apenas em 2016. Na base que o TSE gera ela aparece erroneamente como NÂO ELEITO

In [66]:
eleitos_supl = deputados_queconcorreram_2014_sp[(deputados_queconcorreram_2014_sp['situacao'] == 'ELEITO POR QP') |
                                           (deputados_queconcorreram_2014_sp['situacao'] == 'ELEITO POR MÉDIA') |
                                           (deputados_queconcorreram_2014_sp['situacao'] == 'ELEITO') |
                                           (deputados_queconcorreram_2014_sp['situacao'] == 'SUPLENTE') |
                                           (deputados_queconcorreram_2014_sp['nome_completo'] == 'VANESSA DAMO OROSCO')]

In [67]:
eleitos_supl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1522 entries, 0 to 2126
Data columns (total 8 columns):
uf                 1522 non-null object
cargo              1522 non-null object
nome_completo      1522 non-null object
sequencial         1522 non-null object
cpf                1522 non-null object
nome_urna          1522 non-null object
partido_eleicao    1522 non-null object
situacao           1522 non-null object
dtypes: object(8)
memory usage: 107.0+ KB


In [68]:
eleitos_supl.to_csv("resultados/deputadoseleitossuplentes.csv", index=False)

Cruza bases

In [69]:
df_tse = pd.merge(eleitos_supl, nomes_deputados_atual_legislatura, left_on='nome_urna', right_on='Deputado')

In [70]:
df_tse.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95 entries, 0 to 94
Data columns (total 9 columns):
uf                 95 non-null object
cargo              95 non-null object
nome_completo      95 non-null object
sequencial         95 non-null object
cpf                95 non-null object
nome_urna          95 non-null object
partido_eleicao    95 non-null object
situacao           95 non-null object
Deputado           95 non-null object
dtypes: object(9)
memory usage: 7.4+ KB


### Verifica os nomes não encontrados na base que veio de despesas, do lado direito

In [71]:
nomes_naoencontrados = pd.merge(eleitos_supl, 
                        nomes_deputados_atual_legislatura, 
                        left_on='nome_urna', 
                        right_on='Deputado',
                        how='outer',
                        indicator=True)

rdf = nomes_naoencontrados.query("_merge == 'right_only'").drop('_merge',axis=1)

rdf.Deputado

1522                   LUIZ CARLOS GONDIM
1523                       EDSON GIRIBONI
1524                       ROQUE BARBIERE
1525                     CELSO NASCIMENTO
1526                      FELICIANO FILHO
1527                      ROBERTO TRIPOLI
1528                 CEZINHA DE MADUREIRA
1529                       CELINO CARDOSO
1530                     PROFESSOR AURIEL
1531                   CARLOS BEZERRA JR.
1532                       TEONILIO BARBA
1533                         CARLOS CEZAR
1534                        AFONSO LOBATO
1535                ALENCAR SANTANA BRAGA
1536                         GILENO GOMES
1537    JORGE WILSON XERIFE DO CONSUMIDOR
1538                         JORGE CARUSO
1539                       DOUTOR ULYSSES
1540                      ORLANDO BOLCONE
1541                   ABELARDO CAMARINHA
1542                      ATILA JACOMUSSI
1543                      JOSE ZICO PRADO
1544                ANTONIO SALIM CURIATI
1545                FRANCISCO CAMP

### Agora é feita checagem manual dos nome_urna no TSE. Se estiverem diferentes dos nomes vindo de despesa serão alterados. Mas se forem deputados NÂO ELEITOS serão descartados

Foram alteradas grafias (exemplo TEONILIO BARBA no TSE está como BARBA) e este nomes não são suplentes no registro do TSE e são da 17ª Legislatura, portanto não vão entrar 16: 

FRANCISCO CAMPOS TITO, 
MARCO AURELIO, 
HAMILTON PEREIRA, 
CONSTANCIA FELIX, 
ADRIANO DIOGO, 
RUI FALCAO, 
HEROILMA SOARES, 
DILMO DOS SANTOS, 
LUIZ CLAUDIO MARCOLINO, 
EDINHO SILVA, 
OLIMPIO GOMES, 
ALEX MANENTE, 
SAMUEL MOREIRA, 
ISAC REIS, 
BRUNO COVAS e 
LUIZ MOURA

### Carrega arquivo do TSE com a grafia dos nomes ajustada

In [76]:
eleitos_supl = pd.read_csv('resultados/deputadoseleitossuplentes.csv', converters={'sequencial': lambda x: str(x), 'cpf': lambda x: str(x)})

### Cruza novamente e checa se ficaram de fora apenas os 16 acima

In [77]:
df_tse = pd.merge(eleitos_supl, nomes_deputados_atual_legislatura, left_on='nome_urna', right_on='Deputado')
df_tse.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120 entries, 0 to 119
Data columns (total 9 columns):
uf                 120 non-null object
cargo              120 non-null object
nome_completo      120 non-null object
sequencial         120 non-null object
cpf                120 non-null object
nome_urna          120 non-null object
partido_eleicao    120 non-null object
situacao           120 non-null object
Deputado           120 non-null object
dtypes: object(9)
memory usage: 9.4+ KB


In [78]:
df_tse.to_csv("resultados/deputadoseleitossuplentes_parcial.csv", index=False)

In [79]:
nomes_naoencontrados = pd.merge(eleitos_supl, 
                        nomes_deputados_atual_legislatura, 
                        left_on='nome_urna', 
                        right_on='Deputado',
                        how='outer',
                        indicator=True)

rdf = nomes_naoencontrados.query("_merge == 'right_only'").drop('_merge',axis=1)

rdf.Deputado

1522     FRANCISCO CAMPOS TITO
1523             MARCO AURELIO
1524          HAMILTON PEREIRA
1525          CONSTANCIA FELIX
1526             ADRIANO DIOGO
1527                RUI FALCAO
1528           HEROILMA SOARES
1529          DILMO DOS SANTOS
1530    LUIZ CLAUDIO MARCOLINO
1531              EDINHO SILVA
1532             OLIMPIO GOMES
1533              ALEX MANENTE
1534            SAMUEL MOREIRA
1535                 ISAC REIS
1536               BRUNO COVAS
1537                LUIZ MOURA
Name: Deputado, dtype: object

### Agora cruzo esses nomes com uma base que tem o código único dos deputados, no caso autores de projetos (IdAutor) - gerado em alesp_projetos

In [80]:
def ajeitar_nomes(nome):
    nome = nome.upper()
    return nome

In [81]:
autores_deputados = pd.read_csv("resultados/todos_autores_projetos.csv", sep=',',encoding = 'utf-8', converters={'IdAutor': lambda x: str(x), 'IdDocumento': lambda x: str(x)})

autores_deputados['NomeAutor'] = autores_deputados['NomeAutor'].apply(ajeitar_nomes)

autores_deputados['NomeAutor'] = autores_deputados['NomeAutor'].apply(f)

In [82]:
autores_deputados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700123 entries, 0 to 700122
Data columns (total 3 columns):
IdAutor        700123 non-null object
IdDocumento    700123 non-null object
NomeAutor      700123 non-null object
dtypes: object(3)
memory usage: 16.0+ MB


In [83]:
df_deputados_codigos = pd.merge(autores_deputados, df_tse, left_on='NomeAutor', right_on='Deputado')

In [84]:
df_deputados_codigos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 345844 entries, 0 to 345843
Data columns (total 12 columns):
IdAutor            345844 non-null object
IdDocumento        345844 non-null object
NomeAutor          345844 non-null object
uf                 345844 non-null object
cargo              345844 non-null object
nome_completo      345844 non-null object
sequencial         345844 non-null object
cpf                345844 non-null object
nome_urna          345844 non-null object
partido_eleicao    345844 non-null object
situacao           345844 non-null object
Deputado           345844 non-null object
dtypes: object(12)
memory usage: 34.3+ MB


Verifica se há nomes não encontrados

In [85]:
nomes_naoencontrados = pd.merge(autores_deputados, 
                        df_tse, 
                        left_on='NomeAutor', 
                        right_on='Deputado',
                        how='outer',
                        indicator=True)
rdf = nomes_naoencontrados.query("_merge == 'right_only'").drop('_merge',axis=1)
rdf.Deputado

700123    LUIZ FERNANDO
Name: Deputado, dtype: object

LUIZ FERNANDO precisa mudar a grafia para LUIZ FERNANDO T. FERREIRA

In [86]:
def limpeza1(linha):
    nome = str(linha["Deputado"])
    if nome == 'LUIZ FERNANDO':
        return 'LUIZ FERNANDO T. FERREIRA'
    else:
        return nome

df_tse['Deputado'] = df_tse.apply(limpeza1, axis=1)

Testa de novo

In [87]:
nomes_naoencontrados = pd.merge(autores_deputados, 
                        df_tse, 
                        left_on='NomeAutor', 
                        right_on='Deputado',
                        how='outer',
                        indicator=True)
rdf = nomes_naoencontrados.query("_merge == 'right_only'").drop('_merge',axis=1)
rdf.Deputado

Series([], Name: Deputado, dtype: object)

In [88]:
df_deputados_codigos = pd.merge(autores_deputados, df_tse, left_on='NomeAutor', right_on='Deputado')

In [89]:
df_deputados_codigos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 347649 entries, 0 to 347648
Data columns (total 12 columns):
IdAutor            347649 non-null object
IdDocumento        347649 non-null object
NomeAutor          347649 non-null object
uf                 347649 non-null object
cargo              347649 non-null object
nome_completo      347649 non-null object
sequencial         347649 non-null object
cpf                347649 non-null object
nome_urna          347649 non-null object
partido_eleicao    347649 non-null object
situacao           347649 non-null object
Deputado           347649 non-null object
dtypes: object(12)
memory usage: 34.5+ MB


## Cria o dataframe final

In [98]:
copia = df_deputados_codigos[['IdAutor','NomeAutor', 'uf', 'nome_completo', 'sequencial', 'cpf', 'nome_urna', 'partido_eleicao', 'situacao']]

In [99]:
selecionados_final = copia.drop_duplicates()

In [100]:
selecionados_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120 entries, 0 to 347646
Data columns (total 9 columns):
IdAutor            120 non-null object
NomeAutor          120 non-null object
uf                 120 non-null object
nome_completo      120 non-null object
sequencial         120 non-null object
cpf                120 non-null object
nome_urna          120 non-null object
partido_eleicao    120 non-null object
situacao           120 non-null object
dtypes: object(9)
memory usage: 9.4+ KB


In [101]:
selecionados_final.rename(columns={"IdAutor": "codigo_unico", "NomeAutor": "nome_deputado"}, inplace=True)

/home/reinaldo/Documentos/Code/volt/alesp/lib/python3.6/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [102]:
selecionados_final.reset_index().head()

,index,codigo_unico,nome_deputado,uf,nome_completo,sequencial,cpf,nome_urna,partido_eleicao,situacao
0,0,31,VAZ DE LIMA,SP,JOSE CARLOS VAZ DE LIMA,250000001566,57339333800,VAZ DE LIMA,PSDB,ELEITO POR QP
1,2412,68,EDMIR CHEDID,SP,EDMIR JOSÉ ABI CHEDID,250000001645,06837351877,EDMIR CHEDID,DEM,ELEITO POR QP
2,17191,39,ALDO DEMARCHI,SP,JOSÉ ALDO DEMARCHI,250000001654,02755653868,ALDO DEMARCHI,DEM,ELEITO POR QP
3,21392,64,ROBERTO ENGLER,SP,ROBERTO CARVALHO ENGLER PINTO,250000001532,01561715891,ROBERTO ENGLER,PSDB,ELEITO POR QP
4,23802,40,LEO OLIVEIRA,SP,OCIMAR DONIZETI LÉO OLIVEIRA,250000002709,07318813897,LEO OLIVEIRA,PMDB,ELEITO POR QP


In [103]:
selecionados_final.to_csv('resultados/deputados_selecionados_final.csv', index=False)